In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv('Movie.csv')

In [3]:
data.head()

,movie_title,num_critic_for_reviews,duration,DIRECTOR_facebook_likes,actor_3_facebook_likes,ACTOR_1_facebook_likes,gross,num_voted_users,Cast_Total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,ACTOR_2_facebook_likes,imdb_score,title_year.1
0,Avatar?ÿ,723,178.0,10,855,1000,760505847,886204.0,4834.0,NaN,3054,237000000,2009,936.0,7.9,2009.0
1,Pirates of the Caribbean: At World's End?ÿ,302,NaN,563,1000,40000,309404152,471220.0,48350.0,NaN,1238,300000000,2007,5000.0,7.1,NaN
2,Spectre?ÿ,602,148.0,20,161,11000,200074175,275868.0,11700.0,1.0,994,245000000,2015,393.0,6.8,2015.0
3,The Dark Knight Rises?ÿ,813,NaN,22000,23000,27000,448130642,1144337.0,106759.0,NaN,2701,250000000,2012,23000.0,8.5,NaN
4,John Carter?ÿ,462,132.0,"""475""",530,640,73058679,212204.0,1873.0,1.0,738,263700000,2012,632.0,6.6,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   movie_title                14 non-null     object 
 1   num_critic_for_reviews     14 non-null     int64  
 2   duration                   11 non-null     float64
 3   DIRECTOR_facebook_likes    12 non-null     object 
 4   actor_3_facebook_likes     14 non-null     int64  
 5   ACTOR_1_facebook_likes     14 non-null     int64  
 6   gross                      14 non-null     int64  
 7   num_voted_users            13 non-null     float64
 8   Cast_Total_facebook_likes  12 non-null     float64
 9   facenumber_in_poster       9 non-null      float64
 10  num_user_for_reviews       14 non-null     int64  
 11  budget                     14 non-null     int64  
 12  title_year                 14 non-null     int64  
 13  ACTOR_2_facebook_likes     13 non-null     float64
 

### Action for special symbols in movie title

In [5]:
def symbols_cleaner(df, col):
    
    """df = dataFrame name,  col =column name"""
    
    df[col] = [re.sub('[^A-Za-z0-9\s]+', '', str(df[col][i])) for i in range(len(df[col]))]

In [6]:
symbols_cleaner(data, 'movie_title')

In [7]:
data.head()

,movie_title,num_critic_for_reviews,duration,DIRECTOR_facebook_likes,actor_3_facebook_likes,ACTOR_1_facebook_likes,gross,num_voted_users,Cast_Total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,ACTOR_2_facebook_likes,imdb_score,title_year.1
0,Avatar,723,178.0,10,855,1000,760505847,886204.0,4834.0,NaN,3054,237000000,2009,936.0,7.9,2009.0
1,Pirates of the Caribbean At Worlds End,302,NaN,563,1000,40000,309404152,471220.0,48350.0,NaN,1238,300000000,2007,5000.0,7.1,NaN
2,Spectre,602,148.0,20,161,11000,200074175,275868.0,11700.0,1.0,994,245000000,2015,393.0,6.8,2015.0
3,The Dark Knight Rises,813,NaN,22000,23000,27000,448130642,1144337.0,106759.0,NaN,2701,250000000,2012,23000.0,8.5,NaN
4,John Carter,462,132.0,"""475""",530,640,73058679,212204.0,1873.0,1.0,738,263700000,2012,632.0,6.6,NaN


In [8]:
# 2 columns has the same name title_years, let's see if those columns containing different values.
data['title_year'].unique()

array([2009, 2007, 2015, 2012, 2010, 2016, 2006, 2008], dtype=int64)

In [9]:
data['title_year.1'].unique()

array([2009.,   nan, 2015., 2007., 2016., 2008.])

### Is [title year.1] a duplicated column?
Last row is not matching in title_year and title_year.1, but i will act like i am dealing with big data by creating new column showing us the matching and not matching values.

In [10]:
# create new column for matching and not matching values in year columns
data['year_matching'] = ['Matching' if data['title_year'][val] == data['title_year.1'][val]
                         else data['title_year.1'][val] for val in range(len(data['title_year']))]

In [11]:
data['year_matching'].value_counts()

Matching    6
2008.0      1
Name: year_matching, dtype: int64

In [12]:
data = data.drop('year_matching', axis=1)

title_year.1 isn't a duplicated column, will interpolate the missing years

In [13]:
data['title_year.1'] = data['title_year.1'].interpolate().astype('int64')

In [14]:
# checking the duplicated movie title
duplicates = data[data.duplicated('movie_title')]

In [15]:
data[data['movie_title'] == duplicates.iloc[0,0]]

,movie_title,num_critic_for_reviews,duration,DIRECTOR_facebook_likes,actor_3_facebook_likes,ACTOR_1_facebook_likes,gross,num_voted_users,Cast_Total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,ACTOR_2_facebook_likes,imdb_score,title_year.1
7,Avengers Age of Ultron,635,141.0,10,19000,26000,458991599,462669.0,92000.0,4.0,1117,250000000,2015,21000.0,7.5,2012
8,Avengers Age of Ultron,635,141.0,10,19000,26000,458991599,462669.0,92000.0,4.0,1117,250000000,2015,21000.0,7.5,2015


In [16]:
# All the values is duplicated except last value is null, so will drop that row, then reset the index
data.drop(index=data.index[7], axis=0, inplace=True)
data.reset_index().drop('index', axis=1)

,movie_title,num_critic_for_reviews,duration,DIRECTOR_facebook_likes,actor_3_facebook_likes,ACTOR_1_facebook_likes,gross,num_voted_users,Cast_Total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,ACTOR_2_facebook_likes,imdb_score,title_year.1
0,Avatar,723,178.0,10,855,1000,760505847,886204.0,4834.0,NaN,3054,237000000,2009,936.0,7.9,2009
1,Pirates of the Caribbean At Worlds End,302,NaN,563,1000,40000,309404152,471220.0,48350.0,NaN,1238,300000000,2007,5000.0,7.1,2012
2,Spectre,602,148.0,20,161,11000,200074175,275868.0,11700.0,1.0,994,245000000,2015,393.0,6.8,2015
3,The Dark Knight Rises,813,NaN,22000,23000,27000,448130642,1144337.0,106759.0,NaN,2701,250000000,2012,23000.0,8.5,2012
4,John Carter,462,132.0,"""475""",530,640,73058679,212204.0,1873.0,1.0,738,263700000,2012,632.0,6.6,2009
5,SpiderMan 3,392,156.0,23,4000,24000,336530303,383056.0,46055.0,NaN,1902,258000000,2007,11000.0,6.2,2007
6,Tangled,324,NaN,15,284,799,200807262,294810.0,NaN,1.0,387,260000000,2010,553.0,7.8,2009
7,Avengers Age of Ultron,635,141.0,10,19000,26000,458991599,462669.0,92000.0,4.0,1117,250000000,2015,21000.0,7.5,2015
8,Harry Potter and the HalfBlood Prince,375,153.0,282,10000,25000,301956980,321795.0,58753.0,3.0,973,250000000,2009,11000.0,7.5,2015
9,Batman v Superman Dawn of Justice,673,183.0,NaN,2000,15000,330249062,NaN,24450.0,NaN,3018,250000000,2016,NaN,6.9,2016


In [17]:
# DIRECTOR_facebook_likes is object type while it's containing integers
data.DIRECTOR_facebook_likes.unique()

array(['10', '563', '20', '22000', '"475"', '23', '15', '282', nan, '395'],
      dtype=object)

In [18]:
def quote_remover(df, col):
    
    """df = DataFrame name, col = column name inside quotes"""
    
    for i in range(len(df[col])):
        try:
            df[col][i] = str(df[col][i]).replace('"','')
        except (KeyError, ValueError):
            np.nan
    df[col] = df[col].astype('float64')

In [19]:
quote_remover(data, 'DIRECTOR_facebook_likes') # removing the quote from some values, and changing the type to float

<ipython-input-18-511bdc582ab1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][i] = str(df[col][i]).replace('"','')


### Action for remaining null values:

will predict the nulls using MultiOutputRegressor

In [20]:
data.head()

,movie_title,num_critic_for_reviews,duration,DIRECTOR_facebook_likes,actor_3_facebook_likes,ACTOR_1_facebook_likes,gross,num_voted_users,Cast_Total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,ACTOR_2_facebook_likes,imdb_score,title_year.1
0,Avatar,723,178.0,10.0,855,1000,760505847,886204.0,4834.0,NaN,3054,237000000,2009,936.0,7.9,2009
1,Pirates of the Caribbean At Worlds End,302,NaN,563.0,1000,40000,309404152,471220.0,48350.0,NaN,1238,300000000,2007,5000.0,7.1,2012
2,Spectre,602,148.0,20.0,161,11000,200074175,275868.0,11700.0,1.0,994,245000000,2015,393.0,6.8,2015
3,The Dark Knight Rises,813,NaN,22000.0,23000,27000,448130642,1144337.0,106759.0,NaN,2701,250000000,2012,23000.0,8.5,2012
4,John Carter,462,132.0,475.0,530,640,73058679,212204.0,1873.0,1.0,738,263700000,2012,632.0,6.6,2009


In [21]:
nulls = data.isnull().sum()>0 # null columns
nulls_col = data.columns[nulls]
data[nulls_col]

,duration,DIRECTOR_facebook_likes,num_voted_users,Cast_Total_facebook_likes,facenumber_in_poster,ACTOR_2_facebook_likes
0,178.0,10.0,886204.0,4834.0,NaN,936.0
1,NaN,563.0,471220.0,48350.0,NaN,5000.0
2,148.0,20.0,275868.0,11700.0,1.0,393.0
3,NaN,22000.0,1144337.0,106759.0,NaN,23000.0
4,132.0,475.0,212204.0,1873.0,1.0,632.0
5,156.0,23.0,383056.0,46055.0,NaN,11000.0
6,NaN,15.0,294810.0,NaN,1.0,553.0
8,141.0,10.0,462669.0,92000.0,4.0,21000.0
9,153.0,282.0,321795.0,58753.0,3.0,11000.0
10,183.0,NaN,NaN,24450.0,NaN,NaN


In [22]:
data['duration'] = data['duration'].fillna(np.mean(data['duration'])).astype('int64')

In [23]:
data[nulls_col] = data[nulls_col].fillna((data[nulls_col].mean())).astype('int64')

In [23]:
# changing some columns name
data.columns

Index(['movie_title', 'num_critic_for_reviews', 'duration',
       'DIRECTOR_facebook_likes', 'actor_3_facebook_likes',
       'ACTOR_1_facebook_likes', 'gross', 'num_voted_users',
       'Cast_Total_facebook_likes', 'facenumber_in_poster',
       'num_user_for_reviews', 'budget', 'title_year',
       'ACTOR_2_facebook_likes', 'imdb_score', 'title_year.1'],
      dtype='object')

In [27]:
data.rename(columns = {'DIRECTOR_facebook_likes':'director_facebook_likes', 
                       'ACTOR_1_facebook_likes':'actor_1_facebook_likes',
                       'Cast_Total_facebook_likes':'cast_total_facebook_likes',
                       'ACTOR_2_facebook_likes':'actor_2_facebook_likes'})

,movie_title,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,cast_total_facebook_likes,facenumber_in_poster,num_user_for_reviews,budget,title_year,actor_2_facebook_likes,imdb_score,title_year.1
0,Avatar,723,178,10,855,1000,760505847,886204,4834,1,3054,237000000,2009,936,7.9,2009
1,Pirates of the Caribbean At Worlds End,302,151,563,1000,40000,309404152,471220,48350,1,1238,300000000,2007,5000,7.1,2012
2,Spectre,602,148,20,161,11000,200074175,275868,11700,1,994,245000000,2015,393,6.8,2015
3,The Dark Knight Rises,813,151,22000,23000,27000,448130642,1144337,106759,1,2701,250000000,2012,23000,8.5,2012
4,John Carter,462,132,475,530,640,73058679,212204,1873,1,738,263700000,2012,632,6.6,2009
5,SpiderMan 3,392,156,23,4000,24000,336530303,383056,46055,1,1902,258000000,2007,11000,6.2,2007
6,Tangled,324,151,15,284,799,200807262,294810,40480,1,387,260000000,2010,553,7.8,2009
8,Avengers Age of Ultron,635,141,10,19000,26000,458991599,462669,92000,4,1117,250000000,2015,21000,7.5,2015
9,Harry Potter and the HalfBlood Prince,375,153,282,10000,25000,301956980,321795,58753,3,973,250000000,2009,11000,7.5,2015
10,Batman v Superman Dawn of Justice,673,183,2214,2000,15000,330249062,462115,24450,1,3018,250000000,2016,7410,6.9,2016


In [28]:
outputfile = 'cleaned_df.csv'
data.to_csv(outputfile, index=False)